In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random
import statistics
from math import pi
from matplotlib.gridspec import GridSpec
from dateutil.relativedelta import *
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import seaborn as sns
import scipy.stats
import pingouin as pg
import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [17]:
# COMMON VARIABLES

alg_to_analyze = [
    'vader_sentiment_analysis_score',
    'vader_sentiment_analysis_polarity'
]

tpd_columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]

In [18]:
!python3 '/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/main_generate_json_files.py'

Starting script...
Removing existing file...
Ending script...


In [19]:
data = pd.read_json(r'/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/alldata.json')
data.head()

,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
id_str_twitter,23781146,15148309,927028057763995648,77648005,3429765093,10794662,1034514325644431361,529445600,12050562,62314491,18289937,172880784,18609547,2164986296,1026783544457273345,109604376,1040939562774790144,69734383,119897261
name,Fe Rebelatto,Sommelier de Airbnb,✪ Felippe,Conference Distinguished Specialist,Paula Santana,Maharaja of the Legacies,Jessica,"var nickname = ""Riquettinha"";",Roberta Arcoverde,Gabriela,Thamara Andrade,felipe.ts ⚡,Juan Lopes,Rafael Dantas,iagor com i,T̷o̷m̷o̷y̷o̷ ̷ F̷u̷r̷a̷c̷ã̷o̷,Esdras Xavier,belle-🏴.dev || twitch.tv/bellesamways,Willian Justen
screen_name,rebelatto,Guilh_rm_,FelippeRegazio,NannoKa,psanrosa13,rponte,dev_jessi,Riquettinha,rla4,Gabrielathalita,thamyk,felipefialho_,juanplopes,RaffaelDantass,Iagor51,RafaelMansilha,esdras_xavieer,bellesamways,Willian_justen
location,"Rio de Janeiro, Brasil",somewhere in Brazil,,Ctba-PR Brasil,Brasil,"Ceará, Brazil",São Paulo - Brasil,"Belo Horizonte, Brasil","Sao Paulo, Brazil",Belo Horizonte,"Belo Horizonte, Brazil.",Dark Side of the Moon,"Rio de Janeiro, Brasil","São Paulo, Brasil",Pvh,"Palmas, Tocantins, Brasil","Jaraguá do Sul, Brasil",Curitiba,"Petrópolis, Brasil"
url,None,https://t.co/gX4oRZWHg3,https://t.co/KkaPlVH4Wu,https://t.co/hDni4Fh2Ng,https://t.co/YJQuDtv2YW,https://t.co/3yZ94QNQvs,https://t.co/MtUeSEBICu,https://t.co/QU71LJgSuD,None,https://t.co/AmkOKWlftK,https://t.co/JWBzdmQOcV,https://t.co/yPuEuh1bn9,https://t.co/Uhw9rEUK2v,None,None,https://t.co/9cS34v6hKc,https://t.co/H7Mzn0CEHR,https://t.co/bf6AeBboQk,https://t.co/23lnSqtqOj


In [20]:
def clean_df(df, alg):
    clean_df_columns = [
    #     'text',
        'created_at',
        'id_user',
        'final_score',
        'final_polarity',
        'oplexicon_sentiment_analysis_score',
        'oplexicon_sentiment_analysis_polarity',
        'sentistrength_sentiment_analysis_score',
        'sentistrength_sentiment_analysis_polarity',
        'sentilexpt_sentiment_analysis_score',
        'sentilexpt_sentiment_analysis_polarity'
    ]
    if alg == 'oplexicon':
        clean_df_columns.remove('oplexicon_sentiment_analysis_score')
        clean_df_columns.remove('oplexicon_sentiment_analysis_polarity')
    elif alg == 'sentistrength':
        clean_df_columns.remove('sentistrength_sentiment_analysis_score')
        clean_df_columns.remove('sentistrength_sentiment_analysis_polarity')
    elif alg == 'sentilexpt':
        clean_df_columns.remove('sentilexpt_sentiment_analysis_score')
        clean_df_columns.remove('sentilexpt_sentiment_analysis_polarity')
        
    for c in clean_df_columns:
        del df[c]

In [21]:
def create_tweets_pd(data, id_user):

    tweets_pd = pd.DataFrame(data[id_user]['tweets'])
    del tweets_pd[0]
    del tweets_pd[1]
    del tweets_pd[4]
    del tweets_pd[5]
    del tweets_pd[6]
    del tweets_pd[18]
    del tweets_pd[19]
    del tweets_pd[20]
    
    tweets_pd.columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]
 
    return tweets_pd

In [22]:
# VADER X OPLEXICON

tweets = [data[d]['tweets'] for d in data]

final_tweets = []
for i in tweets:
    final_tweets.extend(i)
    
df = pd.DataFrame(final_tweets)

del df[0]
del df[1]
del df[4]
del df[5]
del df[6]
del df[18]
del df[19]
del df[20]

df.columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]

clean_df(df, 'oplexicon')

df['vader_prediction'] = df['vader_sentiment_analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))
df['oplexicon_prediction'] = df['oplexicon_sentiment_analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))

df['accuracy'] = df.apply(lambda x: 1 if x['vader_prediction'] == x['oplexicon_prediction'] else 0, axis=1)

df.head()

,text,vader_sentiment_analysis_score,vader_sentiment_analysis_polarity,oplexicon_sentiment_analysis_score,oplexicon_sentiment_analysis_polarity,vader_prediction,oplexicon_prediction,accuracy
0,"Atenção, estou mudando os moveis da sala de lu...",-0.4588,neg,0.000000,neu,-1,0,0
1,RT @andreariell: o cancelamento dos participan...,0.0000,neu,0.000000,neu,0,0,1
2,RT @AndrewBrobston: NOVA ANTECIPAÇÃO\nBoa tard...,0.7734,pos,0.458831,pos,1,1,1
3,"@DouglasMarendaz Eu amo muito meu pai, mas fal...",-0.8720,neg,0.458831,pos,-1,1,0
4,@lilicebk Eu cresci numa casa cheia de gente e...,-0.6808,neg,0.000000,neu,-1,0,0


In [23]:
def conf_matrix(x, col1, col2):
    if x[col1] == 1 and x[col2] == 1:
        return 'TP'
    elif x[col1] == 1 and x[col2] == -1:
        return 'FN'
    elif x[col1] == -1 and x[col2] == 1:
        return 'FP'
    elif x[col1] == -1 and x[col2] == -1:
        return 'TN'
    else:
        return 0

In [24]:
df['conf_matrix'] = df.apply(lambda x: conf_matrix(x, 'vader_prediction', 'oplexicon_prediction'), axis=1)

list_precision = []

conf_vals = df['conf_matrix'].value_counts().to_dict()
print(conf_vals)

accuracy = (conf_vals['TP'] + conf_vals['TN']) / (conf_vals['TP'] + conf_vals['TN'] + conf_vals['FP'] + conf_vals['FN'])
precision_pos = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FP'])

# ------------------------------------------------------
# verificar se devo incluir os NEUTROS também nas contas
# ------------------------------------------------------

recall_pos = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FN'])
f1_score_pos = 2*precision_pos*recall_pos / (precision_pos + recall_pos)
print('-----------------')
print('VADER X OPLEXICON')
print('-----------------')
print('POSITIVE')
print('--------')
print('Accuracy: ', round(100 * accuracy, 2),'%',
      '\nPrecision: ', round(100 * precision_pos, 2),'%',
      '\nRecall: ', round(100 * recall_pos, 2),'%',
      '\nF1 Score: ', round(100 * f1_score_pos, 2),'%')

print('--------')
print('NEGATIVE')
print('--------')

precision_neg = conf_vals['TN'] / (conf_vals['TN'] + conf_vals['FN'])
recall_neg = conf_vals['TN'] / (conf_vals['TN'] + conf_vals['FP'])
f1_score_neg = 2*precision_neg*recall_neg / (precision_neg + recall_neg)

print('Accuracy: ', round(100 * accuracy, 2),'%',
      '\nPrecision: ', round(100 * precision_neg, 2),'%',
      '\nRecall: ', round(100 * recall_neg, 2),'%',
      '\nF1 Score: ', round(100 * f1_score_neg, 2),'%')

{0: 48939, 'TP': 20534, 'FP': 12911, 'TN': 10195, 'FN': 5197}
-----------------
VADER X OPLEXICON
-----------------
POSITIVE
--------
Accuracy:  62.92 % 
Precision:  61.4 % 
Recall:  79.8 % 
F1 Score:  69.4 %
--------
NEGATIVE
--------
Accuracy:  62.92 % 
Precision:  66.24 % 
Recall:  44.12 % 
F1 Score:  52.96 %


In [25]:
rdf = pd.DataFrame(columns=['Sentiment Lexicon', 'Accuracy', 'Precision - Positive', 
                            'Recall - Positive', 'F1-Score - Positive',
                            'Precision - Negative', 
                            'Recall - Negative', 'F1-Score - Negative'
                           ])
d = {
    'Sentiment Lexicon': 'OpLexicon',
    'Accuracy': accuracy,
    'Precision - Positive': precision_pos,
    'Recall - Positive': recall_pos,
    'F1-Score - Positive': f1_score_pos,
    'Precision - Negative': precision_neg,
    'Recall - Negative': recall_neg,
    'F1-Score - Negative': f1_score_neg
}
rdf = rdf.append(d, ignore_index=True)
rdf

,Sentiment Lexicon,Accuracy,Precision - Positive,Recall - Positive,F1-Score - Positive,Precision - Negative,Recall - Negative,F1-Score - Negative
0,OpLexicon,0.629216,0.613963,0.798026,0.693998,0.662357,0.441227,0.529638


In [26]:
# VADER X SENTISTRENGTH

tweets = [data[d]['tweets'] for d in data]

final_tweets = []
for i in tweets:
    final_tweets.extend(i)
    
df = pd.DataFrame(final_tweets)

del df[0]
del df[1]
del df[4]
del df[5]
del df[6]
del df[18]
del df[19]
del df[20]

df.columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]

clean_df(df, 'sentistrength')

df['vader_prediction'] = df['vader_sentiment_analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))
df['sentistrength_prediction'] = df['sentistrength_sentiment_analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))

df['accuracy'] = df.apply(lambda x: 1 if x['vader_prediction'] == x['sentistrength_prediction'] else 0, axis=1)

df.head()

,text,vader_sentiment_analysis_score,vader_sentiment_analysis_polarity,sentistrength_sentiment_analysis_score,sentistrength_sentiment_analysis_polarity,vader_prediction,sentistrength_prediction,accuracy
0,"Atenção, estou mudando os moveis da sala de lu...",-0.4588,neg,0.00,neu,-1,0,0
1,RT @andreariell: o cancelamento dos participan...,0.0000,neu,0.00,neu,0,0,1
2,RT @AndrewBrobston: NOVA ANTECIPAÇÃO\nBoa tard...,0.7734,pos,0.00,neu,1,0,0
3,"@DouglasMarendaz Eu amo muito meu pai, mas fal...",-0.8720,neg,0.25,pos,-1,1,0
4,@lilicebk Eu cresci numa casa cheia de gente e...,-0.6808,neg,0.00,neu,-1,0,0


In [27]:
df['conf_matrix'] = df.apply(lambda x: conf_matrix(x, 'vader_prediction', 'sentistrength_prediction'), axis=1)

conf_vals = df['conf_matrix'].value_counts().to_dict()
print(conf_vals)

accuracy = (conf_vals['TP'] + conf_vals['TN']) / (conf_vals['TP'] + conf_vals['TN'] + conf_vals['FP'] + conf_vals['FN'])
precision_pos = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FP'])
recall_pos = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FN'])
f1_score_pos = 2*precision_pos*recall_pos / (precision_pos + recall_pos)
print('---------------------')
print('VADER X SENTISTRENGTH')
print('---------------------')
print('POSITIVE')
print('--------')
print('Accuracy: ', round(100 * accuracy, 2),'%',
      '\nPrecision: ', round(100 * precision_pos, 2),'%',
      '\nRecall: ', round(100 * recall_pos, 2),'%',
      '\nF1 Score: ', round(100 * f1_score_pos, 2),'%')

print('--------')
print('NEGATIVE')
print('--------')

precision_neg = conf_vals['TN'] / (conf_vals['TN'] + conf_vals['FN'])
recall_neg = conf_vals['TN'] / (conf_vals['TN'] + conf_vals['FP'])
f1_score_neg = 2*precision_neg*recall_neg / (precision_neg + recall_neg)

print('Accuracy: ', round(100 * accuracy, 2),'%',
      '\nPrecision: ', round(100 * precision_neg, 2),'%',
      '\nRecall: ', round(100 * recall_neg, 2),'%',
      '\nF1 Score: ', round(100 * f1_score_neg, 2),'%')

{0: 53855, 'TP': 18847, 'TN': 15084, 'FP': 5964, 'FN': 4026}
---------------------
VADER X SENTISTRENGTH
---------------------
POSITIVE
--------
Accuracy:  77.25 % 
Precision:  75.96 % 
Recall:  82.4 % 
F1 Score:  79.05 %
--------
NEGATIVE
--------
Accuracy:  77.25 % 
Precision:  78.93 % 
Recall:  71.66 % 
F1 Score:  75.12 %


In [28]:
d = {
    'Sentiment Lexicon': 'SentiStrength',
    'Accuracy': accuracy,
    'Precision - Positive': precision_pos,
    'Recall - Positive': recall_pos,
    'F1-Score - Positive': f1_score_pos,
    'Precision - Negative': precision_neg,
    'Recall - Negative': recall_neg,
    'F1-Score - Negative': f1_score_neg
}
rdf = rdf.append(d, ignore_index=True)
rdf

,Sentiment Lexicon,Accuracy,Precision - Positive,Recall - Positive,F1-Score - Positive,Precision - Negative,Recall - Negative,F1-Score - Negative
0,OpLexicon,0.629216,0.613963,0.798026,0.693998,0.662357,0.441227,0.529638
1,SentiStrength,0.772546,0.759623,0.823985,0.790496,0.789325,0.716648,0.751233


In [29]:
# VADER X SENTILEXPT

tweets = [data[d]['tweets'] for d in data]

final_tweets = []
for i in tweets:
    final_tweets.extend(i)
    
df = pd.DataFrame(final_tweets)

del df[0]
del df[1]
del df[4]
del df[5]
del df[6]
del df[18]
del df[19]
del df[20]

df.columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]

clean_df(df, 'sentilexpt')

df['vader_prediction'] = df['vader_sentiment_analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))
df['sentilexpt_prediction'] = df['sentilexpt_sentiment_analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))

df['accuracy'] = df.apply(lambda x: 1 if x['vader_prediction'] == x['sentilexpt_prediction'] else 0, axis=1)

df.head()

,text,vader_sentiment_analysis_score,vader_sentiment_analysis_polarity,sentilexpt_sentiment_analysis_score,sentilexpt_sentiment_analysis_polarity,vader_prediction,sentilexpt_prediction,accuracy
0,"Atenção, estou mudando os moveis da sala de lu...",-0.4588,neg,0.00,neu,-1,0,0
1,RT @andreariell: o cancelamento dos participan...,0.0000,neu,0.00,neu,0,0,1
2,RT @AndrewBrobston: NOVA ANTECIPAÇÃO\nBoa tard...,0.7734,pos,0.00,neu,1,0,0
3,"@DouglasMarendaz Eu amo muito meu pai, mas fal...",-0.8720,neg,-0.25,neg,-1,-1,1
4,@lilicebk Eu cresci numa casa cheia de gente e...,-0.6808,neg,-0.25,neg,-1,-1,1


In [30]:
df['conf_matrix'] = df.apply(lambda x: conf_matrix(x, 'vader_prediction', 'sentilexpt_prediction'), axis=1)

conf_vals = df['conf_matrix'].value_counts().to_dict()
print(conf_vals)

accuracy = (conf_vals['TP'] + conf_vals['TN']) / (conf_vals['TP'] + conf_vals['TN'] + conf_vals['FP'] + conf_vals['FN'])
precision_pos = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FP'])
recall_pos = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FN'])
f1_score_pos = 2*precision_pos*recall_pos / (precision_pos + recall_pos)
print('------------------')
print('VADER X SENTILEXPT')
print('------------------')
print('POSITIVE')
print('--------')
print('Accuracy: ', round(100 * accuracy, 2),'%',
      '\nPrecision: ', round(100 * precision_pos, 2),'%',
      '\nRecall: ', round(100 * recall_pos, 2),'%',
      '\nF1 Score: ', round(100 * f1_score_pos, 2),'%')

print('--------')
print('NEGATIVE')
print('--------')

precision_neg = conf_vals['TN'] / (conf_vals['TN'] + conf_vals['FN'])
recall_neg = conf_vals['TN'] / (conf_vals['TN'] + conf_vals['FP'])
f1_score_neg = 2*precision_neg*recall_neg / (precision_neg + recall_neg)

print('Accuracy: ', round(100 * accuracy, 2),'%',
      '\nPrecision: ', round(100 * precision_neg, 2),'%',
      '\nRecall: ', round(100 * recall_neg, 2),'%',
      '\nF1 Score: ', round(100 * f1_score_neg, 2),'%')

{0: 54017, 'TP': 16448, 'TN': 14604, 'FP': 6971, 'FN': 5736}
------------------
VADER X SENTILEXPT
------------------
POSITIVE
--------
Accuracy:  70.96 % 
Precision:  70.23 % 
Recall:  74.14 % 
F1 Score:  72.14 %
--------
NEGATIVE
--------
Accuracy:  70.96 % 
Precision:  71.8 % 
Recall:  67.69 % 
F1 Score:  69.68 %


In [31]:
d = {
    'Sentiment Lexicon': 'Sentilex-PT',
    'Accuracy': accuracy,
    'Precision - Positive': precision_pos,
    'Recall - Positive': recall_pos,
    'F1-Score - Positive': f1_score_pos,
    'Precision - Negative': precision_neg,
    'Recall - Negative': recall_neg,
    'F1-Score - Negative': f1_score_neg
}
rdf = rdf.append(d, ignore_index=True)
rdf

,Sentiment Lexicon,Accuracy,Precision - Positive,Recall - Positive,F1-Score - Positive,Precision - Negative,Recall - Negative,F1-Score - Negative
0,OpLexicon,0.629216,0.613963,0.798026,0.693998,0.662357,0.441227,0.529638
1,SentiStrength,0.772546,0.759623,0.823985,0.790496,0.789325,0.716648,0.751233
2,Sentilex-PT,0.709614,0.702336,0.741435,0.721356,0.717994,0.676895,0.696839


In [32]:
def create_data():
    bar_chart_data = []

    count = 1
    for i in data:
        if data[i]['bigfive']:
            tweets = create_tweets_pd(data, i)

            d = ['P{}'.format(count),
                 len(tweets.query("final_polarity == 'pos'")),
                 len(tweets.query("final_polarity == 'neu'")), 
                 len(tweets.query("final_polarity == 'neg'")),
                 (len(tweets.query("final_polarity == 'pos'")) / len(tweets)) * 100,
                 (len(tweets.query("final_polarity == 'neu'")) / len(tweets)) * 100,
                 (len(tweets.query("final_polarity == 'neg'")) / len(tweets)) * 100,
                 data[i]['bigfive'][0][1],
                 data[i]['bigfive'][0][2],
                 data[i]['bigfive'][0][3],
                 data[i]['bigfive'][0][4],
                 data[i]['bigfive'][0][5]
                ]
            bar_chart_data.append(d)
            count += 1

    df = pd.DataFrame(bar_chart_data, columns=[
        "Participant","pos", "neu", "neg", 
        "por_pos","por_neu","por_neg", "O", "C", "E", "A", "N"])
    
    biglist = ['O', 'C', 'E', 'A', 'N']
    for b in biglist:
        res = []
        for r in df[b]:
            if r <= 10:
                res.append(1)
            elif r > 10 and r < 21:
                res.append(2)
            elif r > 20 and r < 31:
                res.append(3)
            elif r > 30 and r < 41:
                res.append(4)
            elif r > 40:
                res.append(5)
        
        df['{}_Coded'.format(b)] = res
    
    return df

In [33]:
df = create_data()

In [34]:
df

,Participant,pos,neu,neg,por_pos,por_neu,por_neg,O,C,E,A,N,O_Coded,C_Coded,E_Coded,A_Coded,N_Coded
0,P1,1991,736,2311,39.519651,14.608972,45.871378,37,43,36,24,26,4,5,4,3,3
1,P2,2009,717,1822,44.173263,15.765172,40.061566,41,43,34,22,23,5,5,4,3,3
2,P3,2774,598,2560,46.763318,10.080917,43.155765,43,46,31,24,15,5,5,4,3,2
3,P4,2016,338,817,63.576159,10.659098,25.764743,17,33,33,22,39,2,4,4,3,4
4,P5,3342,467,2184,55.765059,7.792424,36.442516,22,33,31,20,8,3,4,4,2,1
5,P6,5575,770,3040,59.403303,8.204582,32.392115,39,49,39,28,15,4,5,4,3,2
6,P7,890,159,496,57.605178,10.291262,32.103560,35,35,32,26,16,4,4,4,3,2
7,P8,1170,478,947,45.086705,18.420039,36.493256,29,45,34,35,29,3,5,4,4,3
8,P9,4284,650,1522,66.356877,10.068154,23.574969,40,46,38,22,21,4,5,4,3,3
9,P10,2369,313,2026,50.318607,6.648258,43.033135,32,46,19,18,28,4,5,2,2,3


In [35]:
type(df['C'])

pandas.core.series.Series

In [36]:
print(list(df['A']))

[24, 22, 24, 22, 20, 28, 26, 35, 22, 18, 33, 31, 22, 23, 27, 20]


In [37]:
pg.normality(df)

,W,pval,normal
pos,0.753768,0.000706,False
neu,0.501669,0.000002,False
neg,0.563833,0.000008,False
por_pos,0.951069,0.506813,True
por_neu,0.845975,0.011894,False
por_neg,0.944352,0.405776,True
O,0.949463,0.481232,True
C,0.868218,0.025544,False
E,0.959798,0.658125,True
A,0.921497,0.178298,True


In [38]:
pg.homoscedasticity(df)

,W,pval,equal_var
levene,1.303816,0.200281,True


In [39]:
pg.anova(dv='pos', between=['O'], data=df).round(3)

,Source,ddof1,ddof2,F,p-unc,np2
0,O,12,3,0.096,0.999,0.277


In [40]:
pg.anova(dv='pos', between=['E'], data=df).round(3)

,Source,ddof1,ddof2,F,p-unc,np2
0,E,11,4,0.426,0.882,0.539


In [41]:
pg.anova(dv='pos', between=['O', 'A'], data=df).round(3)

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1069: RuntimeWarning: divide by zero encountered in double_scalars
  fval_fac1 = ms_fac1 / ms_resid
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1070: RuntimeWarning: divide by zero encountered in double_scalars
  fval_fac2 = ms_fac2 / ms_resid
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1071: RuntimeWarning: divide by zero encountered in double_scalars
  fval_inter = ms_inter / ms_resid


,Source,SS,DF,MS,F,np2
0,O,2.293538e+07,12,1911281.807,-inf,1.0
1,A,7.830044e+07,10,7830044.194,-inf,1.0
2,O * A,-1.857284e+07,120,-154773.677,inf,1.0
3,Residual,0.000000e+00,-127,-0.000,NaN,NaN


In [42]:
pg.anova(dv='pos', between=['O', 'N'], data=df).round(3)

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1069: RuntimeWarning: divide by zero encountered in double_scalars
  fval_fac1 = ms_fac1 / ms_resid
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1070: RuntimeWarning: divide by zero encountered in double_scalars
  fval_fac2 = ms_fac2 / ms_resid
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/pingouin/parametric.py:1071: RuntimeWarning: divide by zero encountered in double_scalars
  fval_inter = ms_inter / ms_resid


,Source,SS,DF,MS,F,np2
0,O,2.293538e+07,12,1911281.807,-inf,1.0
1,N,5.159759e+07,11,4690690.358,-inf,1.0
2,O * N,8.130007e+06,132,61590.961,-inf,1.0
3,Residual,0.000000e+00,-140,-0.000,NaN,NaN


In [43]:
stats.f_oneway(df['pos'], df['O'])

F_onewayResult(statistic=23.094579533020386, pvalue=4.0308158513247024e-05)

In [44]:
model = ols('pos ~ O + C + E + A + N', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O,4.956257e+06,1.0,0.809436,0.389438
C,7.311207e+06,1.0,1.194038,0.300141
E,1.019985e+07,1.0,1.665800,0.225863
A,2.807873e+06,1.0,0.458571,0.513653
N,2.625112e+05,1.0,0.042872,0.840121
Residual,6.123095e+07,10.0,NaN,NaN


In [45]:
model = ols('neg ~ O + C + E + A + N', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O,4.633169e+06,1.0,0.761170,0.403425
C,9.057502e+06,1.0,1.488031,0.250510
E,5.081020e+06,1.0,0.834746,0.382404
A,9.192827e+05,1.0,0.151026,0.705702
N,2.896960e+04,1.0,0.004759,0.946359
Residual,6.086905e+07,10.0,NaN,NaN


In [46]:
model = ols('por_pos ~ O + C + E + A + N', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O,6.855831,1.0,0.056772,0.816484
C,209.923325,1.0,1.738341,0.216747
E,60.666783,1.0,0.502372,0.494651
A,69.349091,1.0,0.574269,0.466050
N,6.648594,1.0,0.055056,0.819223
Residual,1207.607357,10.0,NaN,NaN


In [47]:
model = ols('por_neg ~ O + C + E + A + N', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O,8.077097,1.0,0.147939,0.708566
C,136.761036,1.0,2.504903,0.144575
E,42.280807,1.0,0.774412,0.399510
A,13.328346,1.0,0.244121,0.631918
N,1.276926,1.0,0.023388,0.881494
Residual,545.973337,10.0,NaN,NaN


In [48]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['O'], alpha=0.05)
print(m_comp)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2 meandiff p-adj    lower      upper    reject
----------------------------------------------------------
    17     22   1326.0   0.9 -43979.5395 46631.5395  False
    17     29  1546.25   0.9 -34270.9238 37363.4238  False
    17     30   1148.0   0.9 -44157.5395 46453.5395  False
    17     31    240.0   0.9 -45065.5395 45545.5395  False
    17     32    353.0   0.9 -44952.5395 45658.5395  False
    17     35  -1126.0   0.9 -46431.5395 44179.5395  False
    17     36  -1290.0   0.9 -46595.5395 44015.5395  False
    17     37    -25.0   0.9 -45330.5395 45280.5395  False
    17     39   3559.0   0.9 -41746.5395 48864.5395  False
    17     40   2268.0   0.9 -43037.5395 47573.5395  False
    17     41     -7.0   0.9 -45312.5395 45298.5395  False
    17     43    758.0   0.9 -44547.5395 46063.5395  False
    22     29   220.25   0.9 -35596.9238 36037.4238  False
    22     30   -178.0   0.9 -45483.5395 45127.5395  Fal

In [49]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['C'], alpha=0.05)
print(m_comp)

    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff p-adj    lower      upper    reject
-----------------------------------------------------------
    33     35    -652.0   0.9 -11823.0548 10519.0548  False
    33     36    -423.0   0.9 -14104.6921 13258.6921  False
    33     41    -804.0   0.9 -14485.6921 12877.6921  False
    33     43   -1021.0   0.9 -11218.7312  9176.7312  False
    33     45    1363.0   0.9  -8834.7312 11560.7312  False
    33     46  463.3333   0.9  -9734.3979 10661.0645  False
    33     49    2896.0   0.9 -10785.6921 16577.6921  False
    35     36     229.0   0.9 -13452.6921 13910.6921  False
    35     41    -152.0   0.9 -13833.6921 13529.6921  False
    35     43    -369.0   0.9 -10566.7312  9828.7312  False
    35     45    2015.0   0.9  -8182.7312 12212.7312  False
    35     46 1115.3333   0.9  -9082.3979 11313.0645  False
    35     49    3548.0   0.9 -10133.6921 17229.6921  False
    36     41    -381.0   0.9 -16179.257

In [50]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['E'], alpha=0.05)
print(m_comp)

     Multiple Comparison of Means - Tukey HSD, FWER=0.05     
group1 group2  meandiff  p-adj     lower      upper    reject
-------------------------------------------------------------
    19     24      795.0    0.9 -24528.6187 26118.6187  False
    19     25     -113.0    0.9 -25436.6187 25210.6187  False
    19     27    -1395.0    0.9 -26718.6187 23928.6187  False
    19     28     -494.0    0.9 -25817.6187 24829.6187  False
    19     31   -88.3333    0.9 -20764.9814 20588.3147  False
    19     32    -1479.0    0.9 -26802.6187 23844.6187  False
    19     33     -353.0    0.9 -25676.6187 24970.6187  False
    19     34     -779.5    0.9 -22710.3971 21151.3971  False
    19     36     3741.5    0.9 -18189.3971 25672.3971  False
    19     38     1915.0    0.9 -23408.6187 27238.6187  False
    19     39     3206.0    0.9 -22117.6187 28529.6187  False
    24     25     -908.0    0.9 -26231.6187 24415.6187  False
    24     27    -2190.0    0.9 -27513.6187 23133.6187  False
    24  

In [51]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['N'], alpha=0.05)
print(m_comp)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2 meandiff p-adj    lower      upper    reject
----------------------------------------------------------
     8     15    616.0   0.9 -21993.5754 23225.5754  False
     8     16  -1688.0   0.9 -27795.2888 24419.2888  False
     8     19  -1388.0   0.9 -27495.2888 24719.2888  False
     8     21   -838.0   0.9 -23447.5754 21771.5754  False
     8     23   -362.0   0.9 -26469.2888 25745.2888  False
     8     26    127.0   0.9 -25980.2888 26234.2888  False
     8     28   -296.0   0.9 -22905.5754 22313.5754  False
     8     29  -1237.0   0.9 -27344.2888 24870.2888  False
     8     32   3699.5   0.9 -18910.0754 26309.0754  False
     8     35   -955.0   0.9 -27062.2888 25152.2888  False
     8     39  -1367.0   0.9 -27474.2888 24740.2888  False
    15     16  -2304.0   0.9 -24913.5754 20305.5754  False
    15     19  -2004.0   0.9 -24613.5754 20605.5754  False
    15     21  -1454.0   0.9  -19914.641  17006.641  Fal

In [52]:
model = ols(formula='pos ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O_Coded,1.591437e+05,1.0,0.025745,0.875720
C_Coded,3.519870e+06,1.0,0.569412,0.467891
E_Coded,1.082742e+07,1.0,1.751561,0.215140
A_Coded,1.073274e+07,1.0,1.736245,0.217004
N_Coded,4.057306e+06,1.0,0.656354,0.436705
Residual,6.181584e+07,10.0,NaN,NaN


In [53]:
model = ols('neg ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O_Coded,2.932008e+05,1.0,0.050270,0.827109
C_Coded,7.751377e+06,1.0,1.328985,0.275798
E_Coded,7.655440e+06,1.0,1.312536,0.278613
A_Coded,6.372682e+06,1.0,1.092606,0.320503
N_Coded,4.296015e+06,1.0,0.736558,0.410864
Residual,5.832555e+07,10.0,NaN,NaN


In [54]:
model = ols('por_pos ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O_Coded,25.099806,1.0,0.282798,0.606484
C_Coded,513.063998,1.0,5.780668,0.037042
E_Coded,31.223039,1.0,0.351789,0.566274
A_Coded,114.485757,1.0,1.289906,0.282552
N_Coded,24.457544,1.0,0.275562,0.611064
Residual,887.551361,10.0,NaN,NaN


In [55]:
model = ols('por_neg ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

,sum_sq,df,F,PR(>F)
O_Coded,0.023721,1.0,0.000526,0.982155
C_Coded,229.837373,1.0,5.095466,0.047585
E_Coded,16.998703,1.0,0.376859,0.552997
A_Coded,17.166701,1.0,0.380584,0.551077
N_Coded,5.596539,1.0,0.124075,0.731967
Residual,451.062509,10.0,NaN,NaN


In [56]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['O_Coded'], alpha=0.05)
print(m_comp)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff  p-adj   lower      upper   reject
----------------------------------------------------------
     2      3  1443.1667   0.9 -6776.7384 9663.0717  False
     2      4   568.4286   0.9 -7567.1675 8704.0247  False
     2      5      375.5   0.9 -8944.9962 9695.9962  False
     3      4  -874.7381   0.9 -5108.6365 3359.1604  False
     3      5 -1067.6667   0.9 -7281.3308 5145.9975  False
     4      5  -192.9286   0.9 -6294.6257 5908.7685  False
----------------------------------------------------------


In [57]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['C_Coded'], alpha=0.05)
print(m_comp)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05   
group1 group2 meandiff p-adj    lower      upper   reject
---------------------------------------------------------
     4      5 755.2182 0.5774 -2022.2042 3532.6406  False
---------------------------------------------------------


In [58]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['E_Coded'], alpha=0.05)
print(m_comp)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff p-adj    lower      upper   reject
----------------------------------------------------------
     2      3   -301.75    0.9 -7564.4724 6960.9724  False
     2      4  813.4545    0.9 -5971.3715 7598.2806  False
     3      4 1115.2045 0.7136 -2677.6285 4908.0376  False
----------------------------------------------------------


In [59]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['A_Coded'], alpha=0.05)
print(m_comp)

    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff  p-adj    lower      upper   reject
-----------------------------------------------------------
     2      3   303.4667    0.9   -3854.89 4461.8234  False
     2      4 -1574.3333 0.6969   -6732.14 3583.4734  False
     3      4    -1877.8 0.4797 -6036.1567 2280.5567  False
-----------------------------------------------------------


In [60]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['N_Coded'], alpha=0.05)
print(m_comp)

    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff  p-adj    lower      upper   reject
-----------------------------------------------------------
     1      2    -850.75    0.9 -9061.8938 7360.3938  False
     1      3 -1062.8571    0.9 -8914.2123 6788.4981  False
     1      4      754.0    0.9 -7457.1438 8965.1438  False
     2      3  -212.1071    0.9 -4815.3722 4391.1579  False
     2      4    1604.75 0.7786 -3588.4333 6797.9333  False
     3      4  1816.8571  0.643 -2786.4079 6420.1222  False
-----------------------------------------------------------


In [61]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['O_Coded'], alpha=0.05)
print(m_comp)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2 meandiff p-adj    lower      upper    reject
----------------------------------------------------------
     2      3   2162.5 0.8307  -5668.756   9993.756  False
     2      4    806.0    0.9 -6944.9333  8556.9333  False
     2      5   1374.0    0.9 -7505.8096 10253.8096  False
     3      4  -1356.5 0.7355 -5390.2136  2677.2136  False
     3      5   -788.5    0.9 -6708.3731  5131.3731  False
     4      5    568.0    0.9    -5245.2     6381.2  False
----------------------------------------------------------


In [62]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['C_Coded'], alpha=0.05)
print(m_comp)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff p-adj    lower      upper   reject
----------------------------------------------------------
     4      5 1366.5636 0.2858 -1274.6723 4007.7996  False
----------------------------------------------------------


In [63]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['E_Coded'], alpha=0.05)
print(m_comp)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff p-adj    lower      upper   reject
----------------------------------------------------------
     2      3    -639.5    0.9 -7778.9963 6499.9963  False
     2      4  416.2727    0.9 -6253.4357 7085.9811  False
     3      4 1055.7727 0.7297 -2672.7076 4784.2531  False
----------------------------------------------------------


In [64]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['A_Coded'], alpha=0.05)
print(m_comp)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05   
group1 group2 meandiff p-adj    lower      upper   reject
---------------------------------------------------------
     2      3    722.7 0.8897 -3395.2406 4840.6406  False
     2      4   -898.0 0.8893 -6005.6767 4209.6767  False
     3      4  -1620.7 0.5656 -5738.6406 2497.2406  False
---------------------------------------------------------


In [65]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['N_Coded'], alpha=0.05)
print(m_comp)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff p-adj    lower      upper   reject
----------------------------------------------------------
     1      2    -461.0    0.9 -8486.0752 7564.0752  False
     1      3 -534.2857    0.9 -8207.7253 7139.1539  False
     1      4   1269.25    0.9 -6755.8252 9294.3252  False
     2      3  -73.2857    0.9 -4572.2385  4425.667  False
     2      4   1730.25 0.7282 -3345.2532 6805.7532  False
     3      4 1803.5357 0.6332  -2695.417 6302.4885  False
----------------------------------------------------------


In [66]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_pos'], groups=df['O_Coded'], alpha=0.05)
print(m_comp)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     2      3 -15.8436 0.5409 -50.3434 18.6563  False
     2      4 -11.7205 0.7245 -45.8665 22.4255  False
     2      5 -18.1079 0.5351  -57.227 21.0113  False
     3      4   4.1231    0.9  -13.647 21.8932  False
     3      5  -2.2643    0.9 -28.3437 23.8151  False
     4      5  -6.3874  0.873 -31.9969 19.2221  False
-----------------------------------------------------


In [67]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_pos'], groups=df['C_Coded'], alpha=0.05)
print(m_comp)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower   upper  reject
---------------------------------------------------
     4      5 -13.7116  0.01 -23.594 -3.8292   True
---------------------------------------------------


In [68]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_pos'], groups=df['E_Coded'], alpha=0.05)
print(m_comp)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj  lower    upper  reject
----------------------------------------------------
     2      3  -0.2546   0.9 -33.6746 33.1653  False
     2      4  -0.0165   0.9 -31.2373 31.2044  False
     3      4   0.2382   0.9 -17.2148 17.6912  False
----------------------------------------------------


In [69]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_pos'], groups=df['A_Coded'], alpha=0.05)
print(m_comp)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     2      3   -6.889 0.5935 -25.2698 11.4917  False
     2      4 -11.8051 0.3868 -34.6037 10.9934  False
     3      4  -4.9161  0.753 -23.2968 13.4647  False
-----------------------------------------------------


In [70]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_pos'], groups=df['N_Coded'], alpha=0.05)
print(m_comp)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj  lower    upper  reject
----------------------------------------------------
     1      2  -6.6758   0.9 -45.2382 31.8866  False
     1      3  -6.4214   0.9 -43.2941 30.4513  False
     1      4  -4.1725   0.9 -42.7349 34.3899  False
     2      3   0.2544   0.9 -21.3641 21.8729  False
     2      4   2.5033   0.9 -21.8857 26.8923  False
     3      4   2.2489   0.9 -19.3696 23.8674  False
----------------------------------------------------


In [71]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_neg'], groups=df['O_Coded'], alpha=0.05)
print(m_comp)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     2      3  12.5363 0.3552  -9.0751 34.1476  False
     2      4   8.8592 0.6122 -12.5305 30.2489  False
     2      5  15.8439 0.2709  -8.6611 40.3489  False
     3      4  -3.6771 0.7449 -14.8086  7.4545  False
     3      5   3.3077    0.9  -13.029 19.6443  False
     4      5   6.9847 0.5786  -9.0576  23.027  False
-----------------------------------------------------


In [72]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_neg'], groups=df['C_Coded'], alpha=0.05)
print(m_comp)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower   upper  reject
---------------------------------------------------
     4      5   8.5198 0.0199 1.5624 15.4772   True
---------------------------------------------------


In [73]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_neg'], groups=df['E_Coded'], alpha=0.05)
print(m_comp)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     2      3  -7.1668 0.6613 -28.9395  14.606  False
     2      4  -7.1552 0.6277 -27.4953 13.1848  False
     3      4   0.0116    0.9 -11.3589  11.382  False
-----------------------------------------------------


In [74]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_neg'], groups=df['A_Coded'], alpha=0.05)
print(m_comp)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     2      3   1.2033    0.9 -11.8327 14.2394  False
     2      4  -1.9794    0.9 -18.1487 14.1899  False
     3      4  -3.1827 0.7877 -16.2188  9.8533  False
-----------------------------------------------------


In [75]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_neg'], groups=df['N_Coded'], alpha=0.05)
print(m_comp)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj  lower    upper  reject
----------------------------------------------------
     1      2  -0.5979   0.9 -26.8543 25.6585  False
     1      3   0.5002   0.9 -24.6057 25.6062  False
     1      4  -0.7587   0.9 -27.0151 25.4977  False
     2      3   1.0981   0.9 -13.6215 15.8178  False
     2      4  -0.1608   0.9 -16.7668 16.4452  False
     3      4  -1.2589   0.9 -15.9786 13.4607  False
----------------------------------------------------


In [76]:
pg.ttest(df['pos'], df['O'])

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,4.805682,30,two-sided,0.00004,"[1621.8, 4018.95]",1.699065,460.826,0.996395


In [77]:
pg.ttest(df['pos'], df['C'])

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,4.790249,30,two-sided,0.000042,"[1612.74, 4009.89]",1.693609,443.843,0.996231


In [78]:
pg.ttest(df['pos'], df['E'])

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,4.807927,30,two-sided,0.00004,"[1623.11, 4020.26]",1.699859,463.351,0.996418


In [79]:
pg.ttest(df['pos'], df['A'])

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,4.818685,30,two-sided,0.000039,"[1629.43, 4026.57]",1.703663,475.65,0.996528


In [80]:
pg.ttest(df['pos'], df['N'])

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,4.81973,30,two-sided,0.000039,"[1630.05, 4027.2]",1.704032,476.861,0.996539


In [81]:
print(pg.mwu(df['pos'], df['O']))
print(pg.mwu(df['neg'], df['O']))

     U-val       tail     p-val  RBC  CLES
MWU  256.0  two-sided  0.000002 -1.0   1.0
     U-val       tail     p-val  RBC  CLES
MWU  256.0  two-sided  0.000002 -1.0   1.0


In [82]:
print(pg.mwu(df['pos'], df['C']))
print(pg.mwu(df['neg'], df['C']))

     U-val       tail     p-val  RBC  CLES
MWU  256.0  two-sided  0.000001 -1.0   1.0
     U-val       tail     p-val  RBC  CLES
MWU  256.0  two-sided  0.000001 -1.0   1.0


In [83]:
print(pg.mwu(df['pos'], df['E']))
print(pg.mwu(df['neg'], df['E']))

     U-val       tail     p-val  RBC  CLES
MWU  256.0  two-sided  0.000002 -1.0   1.0
     U-val       tail     p-val  RBC  CLES
MWU  256.0  two-sided  0.000002 -1.0   1.0


In [84]:
print(pg.mwu(df['pos'], df['A']))
print(pg.mwu(df['neg'], df['A']))

     U-val       tail     p-val  RBC  CLES
MWU  256.0  two-sided  0.000002 -1.0   1.0
     U-val       tail     p-val  RBC  CLES
MWU  256.0  two-sided  0.000002 -1.0   1.0


In [85]:
print(pg.mwu(df['pos'], df['N']))
print(pg.mwu(df['neg'], df['N']))

     U-val       tail     p-val  RBC  CLES
MWU  256.0  two-sided  0.000002 -1.0   1.0
     U-val       tail     p-val  RBC  CLES
MWU  256.0  two-sided  0.000002 -1.0   1.0


In [86]:
print(pg.mwu(df['por_pos'], df['O']))
print(pg.mwu(df['por_neg'], df['O']))

     U-val       tail     p-val       RBC      CLES
MWU  239.0  two-sided  0.000031 -0.867188  0.933594
     U-val       tail     p-val       RBC      CLES
MWU  169.0  two-sided  0.126562 -0.320312  0.660156


In [87]:
print(pg.mwu(df['por_pos'], df['C']))
print(pg.mwu(df['por_neg'], df['C']))

     U-val       tail     p-val       RBC      CLES
MWU  186.0  two-sided  0.030015 -0.453125  0.726562
     U-val       tail     p-val       RBC      CLES
MWU   75.0  two-sided  0.047568  0.414062  0.292969


In [88]:
print(pg.mwu(df['por_pos'], df['E']))
print(pg.mwu(df['por_neg'], df['E']))

     U-val       tail     p-val       RBC      CLES
MWU  247.0  two-sided  0.000008 -0.929688  0.964844
     U-val       tail     p-val     RBC     CLES
MWU  184.0  two-sided  0.036359 -0.4375  0.71875


In [89]:
print(pg.mwu(df['por_pos'], df['A']))
print(pg.mwu(df['por_neg'], df['A']))

     U-val       tail     p-val       RBC      CLES
MWU  254.0  two-sided  0.000002 -0.984375  0.992188
     U-val       tail     p-val     RBC     CLES
MWU  232.0  two-sided  0.000094 -0.8125  0.90625


In [90]:
print(pg.mwu(df['por_pos'], df['N']))
print(pg.mwu(df['por_neg'], df['N']))

     U-val       tail     p-val       RBC      CLES
MWU  253.0  two-sided  0.000003 -0.976562  0.988281
     U-val       tail     p-val   RBC   CLES
MWU  224.0  two-sided  0.000317 -0.75  0.875


In [91]:
print(pg.kruskal(data=df, dv='pos', between='O'))
print(pg.kruskal(data=df, dv='pos', between='O_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='O'))
print(pg.kruskal(data=df, dv='por_pos', between='O_Coded'))
print(pg.kruskal(data=df, dv='neg', between='O'))
print(pg.kruskal(data=df, dv='neg', between='O_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='O'))
print(pg.kruskal(data=df, dv='por_neg', between='O_Coded'))

        Source  ddof1          H     p-unc
Kruskal      O     12  10.147059  0.603061
          Source  ddof1         H    p-unc
Kruskal  O_Coded      3  0.101891  0.99161
        Source  ddof1          H     p-unc
Kruskal      O     12  14.426471  0.274307
          Source  ddof1         H     p-unc
Kruskal  O_Coded      3  2.194328  0.533066
        Source  ddof1          H     p-unc
Kruskal      O     12  11.349265  0.499247
          Source  ddof1         H     p-unc
Kruskal  O_Coded      3  2.609244  0.455871
        Source  ddof1          H     p-unc
Kruskal      O     12  13.996324  0.300943
          Source  ddof1         H     p-unc
Kruskal  O_Coded      3  4.091387  0.251764


In [92]:
print(pg.kruskal(data=df, dv='pos', between='C'))
print(pg.kruskal(data=df, dv='pos', between='C_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='C'))
print(pg.kruskal(data=df, dv='por_pos', between='C_Coded'))
print(pg.kruskal(data=df, dv='neg', between='C'))
print(pg.kruskal(data=df, dv='neg', between='C_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='C'))
print(pg.kruskal(data=df, dv='por_neg', between='C_Coded'))

        Source  ddof1         H     p-unc
Kruskal      C      7  5.919118  0.549225
          Source  ddof1         H     p-unc
Kruskal  C_Coded      1  0.028877  0.865063
        Source  ddof1          H     p-unc
Kruskal      C      7  11.595588  0.114667
          Source  ddof1         H     p-unc
Kruskal  C_Coded      1  4.880214  0.027166
        Source  ddof1         H     p-unc
Kruskal      C      7  5.477941  0.601846
          Source  ddof1         H     p-unc
Kruskal  C_Coded      1  2.698396  0.100449
        Source  ddof1         H     p-unc
Kruskal      C      7  8.352941  0.302503
          Source  ddof1         H     p-unc
Kruskal  C_Coded      1  5.393583  0.020211


In [93]:
print(pg.kruskal(data=df, dv='pos', between='E'))
print(pg.kruskal(data=df, dv='pos', between='E_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='E'))
print(pg.kruskal(data=df, dv='por_pos', between='E_Coded'))
print(pg.kruskal(data=df, dv='neg', between='E'))
print(pg.kruskal(data=df, dv='neg', between='E_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='E'))
print(pg.kruskal(data=df, dv='por_neg', between='E_Coded'))

        Source  ddof1         H     p-unc
Kruskal      E     11  8.948529  0.626643
          Source  ddof1         H     p-unc
Kruskal  E_Coded      2  0.424131  0.808912
        Source  ddof1          H     p-unc
Kruskal      E     11  12.985294  0.294288
          Source  ddof1         H     p-unc
Kruskal  E_Coded      2  0.023061  0.988535
        Source  ddof1          H     p-unc
Kruskal      E     11  10.360294  0.498299
          Source  ddof1         H    p-unc
Kruskal  E_Coded      2  0.897393  0.63846
        Source  ddof1          H     p-unc
Kruskal      E     11  13.448529  0.265023
          Source  ddof1         H     p-unc
Kruskal  E_Coded      2  0.593583  0.743199


In [94]:
print(pg.kruskal(data=df, dv='pos', between='A'))
print(pg.kruskal(data=df, dv='pos', between='A_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='A'))
print(pg.kruskal(data=df, dv='por_pos', between='A_Coded'))
print(pg.kruskal(data=df, dv='neg', between='A'))
print(pg.kruskal(data=df, dv='neg', between='A_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='A'))
print(pg.kruskal(data=df, dv='por_neg', between='A_Coded'))

        Source  ddof1          H     p-unc
Kruskal      A     10  12.441176  0.256614
          Source  ddof1         H     p-unc
Kruskal  A_Coded      2  3.289706  0.193041
        Source  ddof1         H     p-unc
Kruskal      A     10  9.805147  0.457752
          Source  ddof1         H     p-unc
Kruskal  A_Coded      2  1.701471  0.427101
        Source  ddof1          H     p-unc
Kruskal      A     10  13.147059  0.215565
          Source  ddof1         H     p-unc
Kruskal  A_Coded      2  3.830882  0.147277
        Source  ddof1          H     p-unc
Kruskal      A     10  10.742647  0.377917
          Source  ddof1         H     p-unc
Kruskal  A_Coded      2  0.819118  0.663943


In [95]:
print(pg.kruskal(data=df, dv='pos', between='N'))
print(pg.kruskal(data=df, dv='pos', between='N_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='N'))
print(pg.kruskal(data=df, dv='por_pos', between='N_Coded'))
print(pg.kruskal(data=df, dv='neg', between='N'))
print(pg.kruskal(data=df, dv='neg', between='N_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='N'))
print(pg.kruskal(data=df, dv='por_neg', between='N_Coded'))

        Source  ddof1          H     p-unc
Kruskal      N     11  13.191176  0.281014
          Source  ddof1         H     p-unc
Kruskal  N_Coded      3  1.572479  0.665646
        Source  ddof1       H     p-unc
Kruskal      N     11  10.875  0.453794
          Source  ddof1         H     p-unc
Kruskal  N_Coded      3  0.149685  0.985271
        Source  ddof1          H     p-unc
Kruskal      N     11  12.904412  0.299619
          Source  ddof1         H     p-unc
Kruskal  N_Coded      3  0.630252  0.889473
        Source  ddof1          H    p-unc
Kruskal      N     11  11.095588  0.43529
          Source  ddof1         H     p-unc
Kruskal  N_Coded      3  0.143382  0.986166
